# Testing different SA methods 1/5
## nltk's naive bayes
This code was partly taken from [here](http://www.laurentluce.com/posts/twitter-sentiment-analysis-using-python-and-nltk/)
TODO also attribute in thesis

In [35]:
# Imports
import nltk
import csv
import pickle
import random
import time

from gensim import corpora

# Ugly hackery, but necessary: stackoverflow.com/questions/4383571/importing-files-from-different-folder
import sys
sys.path.append('../../../')
    
from src.streaming import spark_functions

In [36]:
preprocess = spark_functions.preprocessor()
tokenize = spark_functions.tokenizer()

with open('./../../../data/interim/sanders_hydrated.csv') as csv_file:
    iterator = csv.reader(csv_file, delimiter=',')
    # Load the parts we need and preprocess as well as tokenize the text
    tweets = [(tokenize(preprocess(text)), sentiment) for (topic, sentiment, id, text) in iterator]
    # The tweets are ordered, so shuffle for the training/test split
    random.shuffle(tweets)

In [37]:
tweets[0]

(['along',
  'buzz',
  'four',
  'google',
  'including',
  'jaiku',
  'jaiku',
  'junkyard',
  'liked',
  'services'],
 'neutral')

In [38]:
"There are %d tweets in this dataset" % len(tweets)

'There are 4347 tweets in this dataset'

In [39]:
# Split between training and test set
# 90% training, 10% test
threshold = int(0.9 * len(tweets))
train_tweets = tweets[:threshold]
test_tweets = tweets[threshold:]

In [49]:
# TODO Bring all other Notebooks into this format

3961

In [58]:
# TODO there might be a function like this in nltk?
def extract_features(document):
    document_words = set(document)
    features = {}
    for word in dictionary.token2id:
        features['contains(%s)' % word] = (word in document_words)
    return features

def test(classifier, test_tweets):
    labeled_correctly = 0
    for tweet in test_tweets:
        if classifier.classify(extract_features(tweet[0])) == tweet[1]:
            labeled_correctly+=1
    print("%d/%d = %.2d percent" % (labeled_correctly, len(test_tweets), labeled_correctly/len(test_tweets)*100))

Train and test the model with the dict from the streamed tweets and from the sanders dataset to show the descrepancies

In [61]:
# Use the dictionary we already created for the topic model
# With its own dict, the accuracy on the sanders data improves significantly
dictionary = corpora.Dictionary.load('./../../../data/processed/tweets_sanders.dict')
t0 = time.time()
training_set = nltk.classify.apply_features(extract_features, train_tweets)
classifier = nltk.NaiveBayesClassifier.train(training_set)
print("Feature extraction and training took %d seconds" % (time.time() - t0))
test(classifier, test_tweets)

Feature extraction and training took 49 seconds
322/435 = 74 percent


In [62]:
# Try the same thing with the other dictionary
dictionary = corpora.Dictionary.load('./../../../data/processed/tweets_stream.dict')
t0 = time.time()
training_set = nltk.classify.apply_features(extract_features, train_tweets)
classifier = nltk.NaiveBayesClassifier.train(training_set)
"Feature extraction and training took %d seconds" % (time.time() - t0)
test(classifier, test_tweets)

272/435 = 62 percent


In [65]:
# Some showcasing
tweet = 'Larry is not my friend'
print(classifier.classify(extract_features(tokenize(preprocess(tweet)))))

neutral


In [15]:
# Save the classifier if it's any good
save_classifier = open("./../../models/naive_bayes/nltk_naive_bayes.pickle","wb")
pickle.dump(classifier, save_classifier)
save_classifier.close()